In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
path='/content/drive/MyDrive/BBC News Train.csv'
df = pd.read_csv(path)
df

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [ ]:
stop = stopwords.words('english')
df['Text_woStopwords'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Text1'] = df['Text_woStopwords'].str.replace('[^\w\s]','')
df['Text1'] = df['Text1'].str.lower()
df['Text1'] = df.apply(lambda row: nltk.word_tokenize(row['Text1']), axis=1)
df = df.drop(['Text_woStopwords'], axis=1)
cnt = 0
for i in df['Text1']:
  lst = []
  for j in i:
    lst.append(lemmatizer.lemmatize(j)) 
  df.at[cnt,'Text'] = lst
  cnt = cnt+1
df = df.drop(['Text1'], axis=1)
df


<ipython-input-4-0f266ef8854d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text1'] = df['Text_woStopwords'].str.replace('[^\w\s]','')


,ArticleId,Text,Category
0,1833,"[worldcom, exboss, launch, defence, lawyer, de...",business
1,154,"[german, business, confidence, slide, german, ...",business
2,1101,"[bbc, poll, indicates, economic, gloom, citize...",business
3,1976,"[lifestyle, governs, mobile, choice, faster, b...",tech
4,917,"[enron, boss, 168m, payout, eighteen, former, ...",business
...,...,...,...
1485,857,"[double, eviction, big, brother, model, capric...",entertainment
1486,325,"[dj, double, act, revamp, chart, show, dj, duo...",entertainment
1487,1590,"[weak, dollar, hit, reuters, revenue, medium, ...",business
1488,1587,"[apple, ipod, family, expands, market, apple, ...",tech


In [ ]:
#checking the unique categories
df.Category.unique()

array(['business', 'tech', 'politics', 'sport', 'entertainment'],
      dtype=object)

In [ ]:
#sorting the list of words
for i in df['Text']:
  i = i.sort()
df


,ArticleId,Text,Category
0,1833,"[11bn, 2001, 2001, 2002, 2002, 2004, 57bn, 675...",business
1,154,"[02, 10, 16, 2, 2003, 2004, 2004, 955, 975, ac...",business
2,1101,"[000, 13, 14, 15, 15, 2001, 2004, 2004, 2005, ...",business
3,1976,"[000, 14, 14, 2003, 2003, 25, 29, 4, 45, able,...",tech
4,917,"[10, 13m, 145m, 155m, 168m, 168m, 168m, 18, 20...",business
...,...,...,...
1485,857,"[000, 12, 12, 15, 4, 46, 5, 50, actor, actress...",entertainment
1486,325,"[1, 1, 1, 103, 27yearold, 30, 40, 40, 40, act,...",entertainment
1487,1590,"[102, 105m, 11, 11, 12, 130m, 160m, 198m, 2003...",business
1488,1587,"[10, 10, 12, 200, 2004, 2004, 2005, 22, 25, 30...",tech


In [ ]:
arr = df.to_numpy() #converting the dataframe to numpy arrays
di = {} #final dictionary of list, which 
        #has the keys as the news cateogories and values as the list of dictionary(key=Article Id, Values= list of tokens)
for i in arr:
  li =i[1]
  di_t = {} # creating a temporary dictionary
  di_t[i[0]] = i[1]
  try:
    di[i[2]].append(li)
  except KeyError:
      di[i[2]] = li
#checking the keys of the dictionary
print(di.keys())
print(di['tech'])


dict_keys(['business', 'tech', 'politics', 'sport', 'entertainment'])
['000', '14', '14', '2003', '2003', '25', '29', '4', '45', 'able', 'across', 'activity', 'advisor', 'ageing', 'ago', 'almost', 'alone', 'also', 'also', 'although', 'always', 'always', 'among', 'analogue', 'another', 'bbc', 'behind', 'better', 'birthday', 'bjorn', 'bjorn', 'bjorn', 'bjorn', 'bjorn', 'bjorn', 'bought', 'camera', 'camera', 'camera', 'camera', 'camera', 'camera', 'camera', 'camera', 'camera', 'camera', 'camera', 'capture', 'change', 'change', 'changed', 'chat', 'child', 'chip', 'choice', 'consumer', 'consumer', 'consumer', 'consumer', 'consumer', 'consumer', 'contact', 'contrast', 'convenient', 'could', 'customer', 'desire', 'device', 'diary', 'diarywriting', 'different', 'different', 'different', 'different', 'digital', 'digital', 'digital', 'divide', 'dr', 'dr', 'dr', 'dr', 'dr', 'dr', 'drive', 'dubbed', 'early', 'encouraged', 'end', 'enterprise', 'equipped', 'ericsson', 'ericsson', 'ericsson', 'ericss

In [ ]:
#calculating the term frequency of each of the terms classwise, 
tf_dict = {} #the following dictionary has the keys as the vocab of the dataset (Text column) and the values as a list of size 5
            #where value at the 0th position of the list denotes the number of times the term occured in the class business
            #where value at the 1sh position of the list denotes the number of times the term occured in the class tech
            #where value at the 2nd position of the list denotes the number of times the term occured in the class politics
            #where value at the 3rd position of the list denotes the number of times the term occured in the class sport
            #where value at the 4th position of the list denotes the number of times the term occured in the class entertainement
for key in di.keys():
  for items in di[key]:
    for i in items:
      if tf_dict.get(i) is None:
        tf_dict[i] = [0, 0, 0, 0, 0]
      if key == 'business':
        tf_dict[i][0] = tf_dict[i][0] + 1
      elif key == 'tech':
        tf_dict[i][1] = tf_dict[i][1] + 1
      elif key == 'politics':
        tf_dict[i][2] = tf_dict[i][2] + 1
      elif key == 'sport':
        tf_dict[i][3] = tf_dict[i][3] + 1
      elif key == 'entertainment':
        tf_dict[i][4] = tf_dict[i][4] + 1
print(tf_dict['ageing']) #printing a sample term frequency class wise
print(len(tf_dict.keys())) #printing the number of distinct terms 


[0, 1, 2, 0, 0]
24662


In [ ]:
#calculating class frequency
cf_dict ={}
for key in tf_dict.keys():
  cf =0
  for v in tf_dict[key]:
    if v > 0:
      cf = cf + 1
  cf_dict[key] = cf
print(cf_dict['ageing']) 

2


In [ ]:
#calculating the inverse class frequency
for key in cf_dict.keys():
  cf_dict[key] = math.log(5/cf_dict[key],10)
print(cf_dict['ageing']) 

0.3979400086720376


In [ ]:
#calculating the tf-icf by multipling the ICF values of the term to each class's frequency
for key in tf_dict.keys():
  tf_dict[key][0] = cf_dict[key] * tf_dict[key][0]
  tf_dict[key][1] = cf_dict[key] * tf_dict[key][1]
  tf_dict[key][2] = cf_dict[key] * tf_dict[key][2]
  tf_dict[key][3] = cf_dict[key] * tf_dict[key][3]
  tf_dict[key][4] = cf_dict[key] * tf_dict[key][4]
print(tf_dict['ageing'])

#now the tf_dict has the final tf-icf values which now have to be converted to a dataframe


[0.0, 0.3979400086720376, 0.7958800173440752, 0.0, 0.0]


In [ ]:
myKeys = list(tf_dict.keys())
myKeys.sort()
sorted_tf_dict = {i: tf_dict[i] for i in myKeys}
# sorted_tf_dict.keys()

In [ ]:
#creating a dataframe wrt to the terms' existence in an article corresponding to a category
#creating a dictionary based on the terms and dictionary
tf_icf_dict = {}
for key in sorted_tf_dict.keys():
  li = []
  for i in range(1490):
    if key in df['Text'][i]:
      if df['Category'][i] == 'business':
        li.append(sorted_tf_dict[key][0])
      elif df['Category'][i] == 'tech':
        li.append(sorted_tf_dict[key][1])
      elif df['Category'][i] == 'politics':
        li.append(sorted_tf_dict[key][2])
      elif df['Category'][i] == 'sport':
        li.append(sorted_tf_dict[key][3])
      elif df['Category'][i] == 'entertainment':
        li.append(sorted_tf_dict[key][4])
    else:
      li.append('0')
  tf_icf_dict[key] = li

In [ ]:
if '11bn' in df['Text'][0]:
  print('yes, ->',sorted_tf_dict['11bn'][0])

yes, -> 5.571160121408527


In [ ]:
#creation of dataframe from tf_df_dcit
tf_icf_dft = pd.DataFrame.from_dict(tf_icf_dict)
tf_icf_dft = tf_icf_dft.join(df)
tf_icf_dft

,0,00,000,0001,00051,000acre,000ayear,000bn,000m,000seater,...,zoom,zooropa,zorro,zuluaga,zurich,zutons,zvonareva,ArticleId,Text,Category
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1833,"[11bn, 2001, 2001, 2002, 2002, 2004, 57bn, 675...",business
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,154,"[02, 10, 16, 2, 2003, 2004, 2004, 955, 975, ac...",business
2,0,0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1101,"[000, 13, 14, 15, 15, 2001, 2004, 2004, 2005, ...",business
3,0,0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1976,"[000, 14, 14, 2003, 2003, 25, 29, 4, 45, able,...",tech
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,917,"[10, 13m, 145m, 155m, 168m, 168m, 168m, 18, 20...",business
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,0,0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,857,"[000, 12, 12, 15, 4, 46, 5, 50, actor, actress...",entertainment
1486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,325,"[1, 1, 1, 103, 27yearold, 30, 40, 40, 40, act,...",entertainment
1487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1590,"[102, 105m, 11, 11, 12, 130m, 160m, 198m, 2003...",business
1488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1587,"[10, 10, 12, 200, 2004, 2004, 2005, 22, 25, 30...",tech


In [ ]:
target = tf_icf_dft['Category']
tf_icf_dft = tf_icf_dft.drop(['Category', 'ArticleId', 'Text'], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tf_icf_dft, target, test_size=0.3, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
gnb = MultinomialNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
acc_score = accuracy_score(y_test, y_pred)
print(acc_score)

0.9977628635346756


In [ ]:
# Calculate the probability of each category based on the frequency of documents
# in the training set that belong to that category.
count = y_train.value_counts()
prob_b = count['business']/count.sum()
prob_t = count['tech']/count.sum()
prob_p = count['politics']/count.sum()
prob_s = count['sport']/count.sum()
prob_e = count['entertainment']/count.sum()
from tabulate import tabulate
data = [['count',count['business'], count['tech'], count['politics'], count['sport'], count['entertainment']],
['probability',prob_b,prob_t ,prob_p ,prob_s ,prob_e ]]
print(tabulate(data, headers=["Type of Values:","business", "tech", "politics", "sport","entertainment"]))


Type of Values:      business        tech    politics       sport    entertainment
-----------------  ----------  ----------  ----------  ----------  ---------------
count                228       188         188         245              194
probability            0.2186    0.180249    0.180249    0.234899         0.186002


In [ ]:
#Calculate the probability of each feature given each category based on the
# TF-ICF values of that feature in documents belonging to that category.
#formula applied is 
df_iii = pd.DataFrame.from_dict(sorted_tf_dict)
df_iii.head() #where row 0= business, row 1 = tech, row2 = politics, row3 = sport, row4 = entertainment
# considering: the probability of a feature given a category = df_iii[term][row number as per category] 
#.e.g. the probability of zombie given a category df_iii['zombie'][1], thus the probability of a feature given each category
# = sum of the values in that feature's column
prob_0 = df_iii.sum()/df_iii.iloc[0].sum()
prob_1 = df_iii.sum()/df_iii.iloc[1].sum()
prob_2 = df_iii.sum()/df_iii.iloc[2].sum()
prob_3 = df_iii.sum()/df_iii.iloc[3].sum()
prob_4 = df_iii.sum()/df_iii.iloc[4].sum()
prob_0, prob_1, prob_2, prob_3, prob_4

(0            0.000000
 00           0.000272
 000          0.000000
 0001         0.000080
 00051        0.000080
                ...   
 zorro        0.000159
 zuluaga      0.000080
 zurich       0.000318
 zutons       0.000080
 zvonareva    0.000319
 Length: 24662, dtype: float64, 0            0.000000
 00           0.000236
 000          0.000000
 0001         0.000069
 00051        0.000069
                ...   
 zorro        0.000138
 zuluaga      0.000069
 zurich       0.000276
 zutons       0.000069
 zvonareva    0.000277
 Length: 24662, dtype: float64, 0            0.000000
 00           0.000326
 000          0.000000
 0001         0.000096
 00051        0.000096
                ...   
 zorro        0.000191
 zuluaga      0.000096
 zurich       0.000381
 zutons       0.000096
 zvonareva    0.000382
 Length: 24662, dtype: float64, 0            0.000000
 00           0.000236
 000          0.000000
 0001         0.000069
 00051        0.000069
                ...   
 zorro    

In [ ]:
print(classification_report(y_test, y_pred,target_names=['business', 'tech','politics','sport','entertainment']))

               precision    recall  f1-score   support

     business       1.00      0.99      1.00       108
         tech       1.00      1.00      1.00        79
     politics       1.00      1.00      1.00        86
        sport       1.00      1.00      1.00       101
entertainment       0.99      1.00      0.99        73

     accuracy                           1.00       447
    macro avg       1.00      1.00      1.00       447
 weighted avg       1.00      1.00      1.00       447



In [ ]:
# Experiment with different preprocessing techniques and parameters to improve
# the performance of the classifier( including different splits like 60-40,80-20,
# 50-50).

#with different splits:
X_train1, X_test1, y_train1, y_test1 = train_test_split(tf_icf_dft, target, test_size=0.4, random_state=42) #60-40
X_train2, X_test2, y_train2, y_test2 = train_test_split(tf_icf_dft, target, test_size=0.2, random_state=42) #80-20
X_train3, X_test3, y_train3, y_test3 = train_test_split(tf_icf_dft, target, test_size=0.5, random_state=42) #50-50

In [ ]:
#training on each split data:
gnb = MultinomialNB()
y_pred1 = gnb.fit(X_train1, y_train1).predict(X_test1)
y_pred2 = gnb.fit(X_train2, y_train2).predict(X_test2)
y_pred3 = gnb.fit(X_train3, y_train3).predict(X_test3)
acc_score1 = accuracy_score(y_test1, y_pred1)
acc_score2 = accuracy_score(y_test2, y_pred2)
acc_score3 = accuracy_score(y_test3, y_pred3)
print("accuracy with 70-30 split:", acc_score)
print("accuracy with 60-40 split:", acc_score1)
print("accuracy with 80-20 split:", acc_score2)
print("accuracy with 50-50 split:", acc_score3)

accuracy with 70-30 split: 0.9977628635346756
accuracy with 60-40 split: 0.9983221476510067
accuracy with 80-20 split: 1.0
accuracy with 50-50 split: 0.9986577181208054


In [ ]:
# with different preprocessing techniques => removing digits and single length character
cols = [c for c in tf_icf_dft.columns if not c.isdigit()]
tf_icf_dft_wo_digits = tf_icf_dft[cols]
#removing the digits and single length characters
cols1 = [c for c in tf_icf_dft_wo_digits.columns if not len(c) == 1]
tf_icf_dft_new = tf_icf_dft_wo_digits[cols1]
tf_icf_dft_new.columns

Index(['000acre', '000ayear', '000bn', '000m', '000seater', '000strong',
       '000th', '000vote', '001and', '001st',
       ...
       'zombie', 'zone', 'zonealarm', 'zoom', 'zooropa', 'zorro', 'zuluaga',
       'zurich', 'zutons', 'zvonareva'],
      dtype='object', length=23771)

In [ ]:
X_trainp, X_testp, y_trainp, y_testp = train_test_split(tf_icf_dft, target, test_size=0.3, random_state=42) #70-30
y_predp = gnb.fit(X_trainp, y_trainp).predict(X_testp)
acc_scorep = accuracy_score(y_testp, y_predp)
print("accuracy with 70-30 split:", acc_score)
print("accuracy with 60-40 split:", acc_score1)
print("accuracy with 80-20 split:", acc_score2)
print("accuracy with 50-50 split:", acc_score3)
print("accuracy with 70-30 split on differently preprocessed data:", acc_scorep)

#trying it on 50-50 split
X_trainp1, X_testp1, y_trainp1, y_testp1 = train_test_split(tf_icf_dft, target, test_size=0.5, random_state=42) #50-50
y_predp1 = gnb.fit(X_trainp1, y_trainp1).predict(X_testp1)
acc_scorep1 = accuracy_score(y_testp1, y_predp1)
print("accuracy with 50-50 split on differently preprocessed data:", acc_scorep1)
#see no difference

accuracy with 70-30 split: 0.9977628635346756
accuracy with 60-40 split: 0.9983221476510067
accuracy with 80-20 split: 1.0
accuracy with 50-50 split: 0.9986577181208054
accuracy with 70-30 split on differently preprocessed data: 0.9977628635346756
accuracy with 50-50 split on differently preprocessed data: 0.9986577181208054


In [ ]:
path='/content/drive/MyDrive/BBC News Train.csv'
df = pd.read_csv(path)
df

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [ ]:
stop = stopwords.words('english')
df['Text_woStopwords'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Text1'] = df['Text_woStopwords'].str.replace('[^\w\s]','')
df['Text1'] = df['Text1'].str.lower()
df['Text1'] = df.apply(lambda row: nltk.word_tokenize(row['Text1']), axis=1)
df = df.drop(['Text_woStopwords'], axis=1)
cnt = 0
for i in df['Text1']:
  lst = []
  for j in i:
    lst.append(lemmatizer.lemmatize(j)) 
  df.at[cnt,'Text'] = lst
  cnt = cnt+1
df = df.drop(['Text1'], axis=1)
# df.assign(Text=df["Text"].apply(lambda l: " ".join(l)))
df['Text']=df["Text"].apply(" ".join)
df


<ipython-input-51-8560e38ead95>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text1'] = df['Text_woStopwords'].str.replace('[^\w\s]','')


,ArticleId,Text,Category
0,1833,worldcom exboss launch defence lawyer defendin...,business
1,154,german business confidence slide german busine...,business
2,1101,bbc poll indicates economic gloom citizen majo...,business
3,1976,lifestyle governs mobile choice faster better ...,tech
4,917,enron boss 168m payout eighteen former enron d...,business
...,...,...,...
1485,857,double eviction big brother model caprice holb...,entertainment
1486,325,dj double act revamp chart show dj duo jk joel...,entertainment
1487,1590,weak dollar hit reuters revenue medium group r...,business
1488,1587,apple ipod family expands market apple expande...,tech


In [ ]:
targ = df['Category']
df = df.drop(['Category'], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, targ, test_size=0.3, random_state=42) #70-30
X_train1, X_test1, y_train1, y_test1 = train_test_split(df, targ, test_size=0.4, random_state=42) #60-40
X_train2, X_test2, y_train2, y_test2 = train_test_split(df, targ, test_size=0.2, random_state=42) #80-20
X_train3, X_test3, y_train3, y_test3 = train_test_split(df, targ, test_size=0.5, random_state=42) #50-50

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
train_X = X_train['Text']  
train_y = y_train
test_X = X_test['Text']
test_y = y_test
#60-40
train_X1 = X_train1['Text']  
train_y1 = y_train1
test_X1 = X_test1['Text']
test_y1 = y_test1
#80-20
train_X2 = X_train2['Text']  
train_y2 = y_train2
test_X2 = X_test2['Text']
test_y2 = y_test2
#50-50
train_X3 = X_train3['Text']  
train_y3 = y_train3
test_X3 = X_test3['Text']
test_y3 = y_test3

In [ ]:
tf_vectorizer = CountVectorizer() # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_train_tf1 = tf_vectorizer.fit_transform(train_X1)
X_train_tf2 = tf_vectorizer.fit_transform(train_X2)
X_train_tf3 = tf_vectorizer.fit_transform(train_X3)

In [ ]:
X_test_tf = tf_vectorizer.transform(test_X)
X_test_tf1 = tf_vectorizer.transform(test_X1)
X_test_tf2 = tf_vectorizer.transform(test_X2)
X_test_tf3 = tf_vectorizer.transform(test_X3)

In [ ]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
naive_bayes_classifier.fit(X_train_tf1, train_y1)
naive_bayes_classifier.fit(X_train_tf2, train_y2)
naive_bayes_classifier.fit(X_train_tf3, train_y3)

MultinomialNB()

In [ ]:
y_pred = naive_bayes_classifier.predict(X_test_tf)
# compute the performance measures
score1 = accuracy_score(test_y, y_pred)
score2 = accuracy_score(test_y1, y_pred1)
score3 = accuracy_score(test_y2, y_pred2)
score4 = accuracy_score(test_y3, y_pred3)
print("accuracy with tf-idf on 70-30 split:",score1)
print("accuracy with tf-idf on 60-40 split:",score2)
print("accuracy with tf-idf on 80-20 split:",score3)
print("accuracy with tf-idf on 50-50 split:",score4)
print("accuracy with tf-icf on 70-30 split:", acc_score)

accuracy with tf-idf on 70-30 split: 0.9798657718120806
accuracy with tf-idf on 60-40 split: 0.9983221476510067
accuracy with tf-idf on 80-20 split: 1.0
accuracy with tf-idf on 50-50 split: 0.9986577181208054
accuracy with tf-icf on 70-30 split: 0.9977628635346756


In [ ]:
#Comparison of result of each tried 
avg_acc_score_idf = (score1 + score2 + score3 + score4)/4
avg_acc_score_icf = (acc_score + acc_score1 + acc_score2 + acc_score3)/4
print("average accuracy of tf-icf over different splits",avg_acc_score_icf )
print("average accuracy of tf-idf over different splits",avg_acc_score_idf )

average accuracy of tf-icf over different splits 0.9986856823266219
average accuracy of tf-idf over different splits 0.9942114093959731


In [ ]:
#Conclusion: tf-icf and tf-idf weighted Naive-Bayes performed almost similar. each of them provided a 99% average  accuracy
# Multinomial NB is well suited out of all the NB variations coz of it's efficiency in handling high dimensional data with the assumption that 
# the features are conditionally independent given the class label. Moreover,
# MNB can handle feature counts that are zero, which is common in text data when some words do not appear in a particular document.

#tf-icf performed slight better as the tf-icf formula took the category into consideration for term and not just article ID